In [7]:
#!/usr/bin/env python
# coding: utf-8
# In[ ]:
#----------------------------------------------------------
# VERSIÓN 2.0 CODIGO CON VARIABLES PREDICTORAS
# SSS
# 8/11/2024
#----------------------------------------------------------
#----------------------------------------------------------
# VERSIÓN 3.0 CODIGO CON DIVISIÓN DE DATOS EN GRUPO TRAIN-TEST
# SSS
# 10/11/2024
#----------------------------------------------------------

import os
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn

#---------------------------------------------------------------
# Obtención y limpieza de datos
#---------------------------------------------------------------

current_dir = os.getcwd()
file_path = os.path.join(current_dir, "Data", "DATOS2023.xlsx")
file_path1 = os.path.join(current_dir, "Data", "DATOS2024.xlsx")

df_2023 = pd.read_excel(file_path)
df_2024 = pd.read_excel(file_path1)

df_2023 = df_2023.rename(columns={"Unnamed: 0": "DÍA"})
df_2024 = df_2024.rename(columns={"Unnamed: 0": "DÍA"})

df_total = pd.concat([df_2023, df_2024], ignore_index=True)
df_total.columns = df_total.columns.str.strip().str.upper()

df_total.replace('ND', 0, inplace=True)
df_total.replace(['X', 'S'], pd.NA, inplace=True)
df_total['MATERIA ORGANICA MG/L BN'].fillna(df_total['COT  MG/L  BN'])
df_total = df_total.apply(pd.to_numeric, errors='coerce')
df_total.fillna(df_total.mean(), inplace=True)
df_total['FECHA'] = pd.to_datetime(df_total['FECHA'], errors='coerce')
df_total = df_total.loc[:, ~df_total.columns.duplicated()]
df_total.drop(columns=['MES'], inplace=True)
df_total.columns = df_total.columns.str.replace(r'[^A-Za-z0-9_]+', '_', regex=True)
df_total = df_total.loc[:, ~df_total.columns.duplicated()]


#---------------------------------------------------------------
# Preprocesamiento para normalización y estandarización
#---------------------------------------------------------------

standard_scaler = StandardScaler()
scaler = MinMaxScaler()
df_normalized = df_total.copy()


#---------------------------------------------------------------
# Selección de variables importantes sin duplicados
#---------------------------------------------------------------

variables_importantes_dos = ['AL_SO_PPM', 'AL_SO_SOLIDO_PPM', 'PAC_PPM', 'FECL3_PPM']

variables_importantes_captación = list(set([
    'TEMPERATURA_C_BN', 'OXIGENO_DISUELTO_MG_L_O2_BN', 'TURBIEDAD_UNT_BN',
    'COLOR_UPC_BN', 'CONDUCTIVIDAD_US_CM_BN', 'PH_BN', 'MATERIA_ORGANICA_MG_L_BN',
    'NITROGENO_AMONIACAL_G_L_BN', 'MANGANESOS_MG_L_MN_BN', 'ALCALINIDAD_TOTAL_MG_L_CACO3_BN',
    'CLORUROS_MG_L_CL_BN', 'DUREZA_TOTAL_MG_L_CACO3_BN', 'DUREZA_CALCICA_MG_L_CACO3_BN',
    'HIERRO_TOTAL_MG_L_FE_3_BN', 'ALUMINIO_RESIDUAL_MG_L_AL_BN', 'POTENCIAL_REDOX_MV_BN',
    'NITRITOS_MG_L_NO2_BN', 'FOSFATOS_MG_L_BN', 'NITRATOS_MG_L_NO3_BN', 'SULFATOS_MG_L_SO4_BN', 'COT_MG_L_BN'
]))

variables_importantes_cruda = list(set([
    'TEMPERATURA_C_CRU', 'OXIGENO_DISUELTO_MG_L_O2_CRU', 'TURBIEDAD_UNT_CRU', 
    'COLOR_UPC_CRU', 'CONDUCTIVIDAD_US_CM_CRU', 'PH_CRU', 'MATERIA_ORGANICA_MG_L_CRU',
    'NITROGENO_AMONIACAL_G_L_CRU', 'MANGANESOS_MG_L_MN_CRU', 'ALCALINIDAD_TOTAL_MG_L_CACO3_CRU',
    'CLORUROS_MG_L_CL_CRU', 'DUREZA_TOTAL_MG_L_CACO3_CRU', 'DUREZA_CALCICA_MG_L_CACO3_CRU', 
    'HIERRO_TOTAL_MG_L_FE_3_CRU', 'ALUMINIO_RESIDUAL_MG_L_AL_CRU', 'POTENCIAL_REDOX_MV_CRU',
    'NITRITOS_MG_L_NO2_CRU', 'NITRATOS_MG_L_NO3_CRU', 'FOSFATOS_MG_L_CRU', 'SULFATOS_MG_L_SO4_CRU', 
    'COT_MG_L_CRU', 'SOLIDOS_SUSPENDIDOS_MG_L_CRU'
]))

variables_importantes_mez = list(set([
    'OXIGENO_DISUELTO_MG_L_O2_MEZ', 'TEMPERATURA_C_MEZ', 'PH_MEZ', 
    'CLORO_LIBRE_MG_L_CL2_MEZ', 'CLORO_COMBINADO_MG_L_CL2_MEZ', 'CLORO_TOTAL_MG_L_CL2_MEZ'
]))

# Crear una lista única de todas las variables
todas_las_variables = list(set(variables_importantes_captación + variables_importantes_cruda + variables_importantes_mez))

#---------------------------------------------------------------
# Estandarización de las variables
#---------------------------------------------------------------

scaler = StandardScaler()
df_standardized_global = df_total.copy()
df_standardized_global[todas_las_variables] = scaler.fit_transform(df_total[todas_las_variables])

# Definir X e y después de la estandarización
X = df_standardized_global[todas_las_variables]
df_standardized_global['DOSIS_TOTAL'] = (
    df_standardized_global['PAC_PPM'] + df_standardized_global['AL_SO_PPM'] + 
    df_standardized_global['AL_SO_SOLIDO_PPM'] + df_standardized_global['FECL3_PPM']
)
y = df_standardized_global['DOSIS_TOTAL']

#---------------------------------------------------------------
# Clustering con K-Means y división de datos en Train-Test-Validation
#---------------------------------------------------------------

kmeans = KMeans(n_clusters=2, random_state=42)
clust_labels = kmeans.fit_predict(X)

X_cluster_0 = X[clust_labels == 0]
y_cluster_0 = y[clust_labels == 0]
X_cluster_1 = X[clust_labels == 1]
y_cluster_1 = y[clust_labels == 1]

#---------------------------------------------------------------
# División de datos en Test y Train
#---------------------------------------------------------------

# Para el Cluster 0
X_train_full_0, X_test_0, y_train_full_0, y_test_0 = train_test_split(X_cluster_0, y_cluster_0, test_size=0.2, random_state=42)
X_train_0, X_val_0, y_train_0, y_val_0 = train_test_split(X_train_full_0, y_train_full_0, test_size=0.25, random_state=42)

# Para el Cluster 1
X_train_full_1, X_test_1, y_train_full_1, y_test_1 = train_test_split(X_cluster_1, y_cluster_1, test_size=0.2, random_state=42)
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X_train_full_1, y_train_full_1, test_size=0.25, random_state=42)

#print("Cluster 0:")
#print(f"Tamaño de X_train_0: {X_train_0.shape}")
#print(f"Tamaño de X_val_0: {X_val_0.shape}")
#print(f"Tamaño de X_test_0: {X_test_0.shape}")

#print("\nCluster 1:")
#print(f"Tamaño de X_train_1: {X_train_1.shape}")
#print(f"Tamaño de X_val_1: {X_val_1.shape}")
#print(f"Tamaño de X_test_1: {X_test_1.shape}")



#---------------------------------------------------------------
# Entrenamiento Modelos de Regresión
#---------------------------------------------------------------

experiment = mlflow.set_experiment("Regresion-DosisOptima")

with mlflow.start_run(experiment_id=experiment.experiment_id):
    n_estimators_0=250
    n_estimators_1=800
    max_depth_0=36
    max_depth_1=48
    
    model_0 = RandomForestRegressor(n_estimators=n_estimators_0, max_depth=max_depth_0, random_state=42)
    # model_0.fit(X_cluster_0, y_cluster_0)
    model_0.fit(X_train_0, y_train_0)
    
    model_1 = RandomForestRegressor(n_estimators=n_estimators_1, max_depth=max_depth_1, random_state=42)
    # model_1.fit(X_cluster_1, y_cluster_1)
    model_1.fit(X_train_1, y_train_1)


    #y_pred_0 = model_0.predict(X_cluster_0)
    #y_pred_1 = model_1.predict(X_cluster_1)

    y_pred_0 = model_0.predict(X_val_0)
    y_pred_1 = model_1.predict(X_val_1)

    mse_0 = mean_squared_error(y_val_0, y_pred_0)
    mae_0 = mean_absolute_error(y_val_0, y_pred_0)
    r2_0 = r2_score(y_val_0, y_pred_0)

    mse_1 = mean_squared_error(y_val_1, y_pred_1)
    mae_1 = mean_absolute_error(y_val_1, y_pred_1)
    r2_1 = r2_score(y_val_1, y_pred_1)

    mlflow.log_param("n_estimators_0", n_estimators_0)
    mlflow.log_param("max_depth_0", max_depth_0)  
    mlflow.log_param("n_estimators_1", n_estimators_1)
    mlflow.log_param("max_depth_1", max_depth_1) 
    mlflow.log_metric("MSE_Cluster_0", mse_0)
    mlflow.log_metric("MAE_Cluster_0", mae_0)
    mlflow.log_metric("R2_Cluster_0", r2_0)
    mlflow.log_metric("MSE_Cluster_1", mse_1)
    mlflow.log_metric("MAE_Cluster_1", mae_1)
    mlflow.log_metric("R2_Cluster_1", r2_1)

    mlflow.sklearn.log_model(model_0, "RandomForestRegressor_Cluster_0")
    mlflow.sklearn.log_model(model_1, "RandomForestRegressor_Cluster_1")

    print("Modelos de Regresión por Clúster - Métricas registradas en MLflow")
    print(f"Cluster 0 - MSE: {mse_0}, MAE: {mae_0}, R2 Score: {r2_0}")
    print(f"Cluster 1 - MSE: {mse_1}, MAE: {mae_1}, R2 Score: {r2_1}")

C:\Users\Sofia\AppData\Local\Temp\ipykernel_31476\3659847527.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_total.replace('ND', 0, inplace=True)
C:\Users\Sofia\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
2024/11/10 11:42:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/10 11:42:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` pa

Modelos de Regresión por Clúster - Métricas registradas en MLflow
Cluster 0 - MSE: 262.7999549090909, MAE: 10.508, R2 Score: 0.6406464704972376
Cluster 1 - MSE: 145.56576564529223, MAE: 9.17810064935065, R2 Score: 0.6521896340662487


In [8]:
X

,ALUMINIO_RESIDUAL_MG_L_AL_BN,POTENCIAL_REDOX_MV_CRU,SULFATOS_MG_L_SO4_CRU,SOLIDOS_SUSPENDIDOS_MG_L_CRU,COT_MG_L_CRU,CLORUROS_MG_L_CL_BN,SULFATOS_MG_L_SO4_BN,CONDUCTIVIDAD_US_CM_BN,OXIGENO_DISUELTO_MG_L_O2_BN,HIERRO_TOTAL_MG_L_FE_3_BN,...,TEMPERATURA_C_MEZ,TURBIEDAD_UNT_BN,PH_BN,COLOR_UPC_CRU,CLORUROS_MG_L_CL_CRU,POTENCIAL_REDOX_MV_BN,NITROGENO_AMONIACAL_G_L_BN,FOSFATOS_MG_L_BN,FOSFATOS_MG_L_CRU,PH_CRU
0,0.100422,-0.193742,-3.365047e-16,0.219631,0.000000,-0.979624,-1.906025e-16,-1.234199,1.354501,0.942723,...,-0.050148,0.336137,0.165949,0.169626,-0.654134,-0.679723,-0.924047,6.027790e-16,-9.559995e-16,0.117516
1,0.100422,0.011687,-1.953100e+00,0.184385,0.000000,-1.235481,-1.588527e+00,-1.208887,1.337060,0.265174,...,-0.050148,0.767566,0.106144,0.110744,-0.695191,0.339489,-0.893020,-1.064505e+00,-2.075557e+00,0.085863
2,4.551616,0.011687,-3.365047e-16,0.262355,0.000000,-0.804094,-1.906025e-16,-1.436691,2.148048,0.759601,...,-0.109443,4.811240,-0.003499,0.300475,-1.115421,0.543331,-1.160429,6.027790e-16,-9.559995e-16,0.075313
3,0.980878,-0.234828,-3.365047e-16,2.378222,0.000000,-1.098627,-1.906025e-16,-1.158264,1.973642,0.832850,...,0.009146,0.763644,0.036371,1.164082,-0.444019,-0.883565,-1.288880,6.027790e-16,-9.559995e-16,0.033109
4,0.344993,-0.111571,-3.365047e-16,1.415881,0.000000,-0.836820,-1.906025e-16,-1.132952,1.197536,0.631417,...,-0.020501,0.567540,0.036371,1.314559,-0.975345,-0.068196,-1.029966,6.027790e-16,-9.559995e-16,0.033109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,0.442821,0.052773,-1.852320e+00,-0.485302,-3.055364,-0.244779,-1.588527e+00,-0.424228,0.159821,-0.236578,...,0.009146,-0.126670,0.076242,-0.353772,0.104212,0.543331,-0.569912,-7.984671e-01,-5.514317e-01,0.054211
543,-0.193064,-0.012964,-3.365047e-16,-0.268481,0.000000,0.240159,-1.906025e-16,-0.398917,-0.023305,-0.434349,...,-0.094619,-0.244333,0.026404,-0.177125,-0.209753,0.135646,0.253907,6.027790e-16,-9.559995e-16,0.001456
544,-0.241978,0.052773,-3.365047e-16,-0.080499,0.000000,0.159832,-1.906025e-16,-0.601409,-0.049466,-0.469142,...,-0.005677,-0.122748,0.036371,-0.164040,-0.477831,0.339489,-0.914417,6.027790e-16,-9.559995e-16,0.022558
545,0.100422,0.011687,-3.365047e-16,-0.481029,14.079324,0.751873,-1.906025e-16,-0.120490,0.194702,-0.286937,...,-0.079796,-0.299242,0.146014,-0.281805,-0.154205,0.339489,0.272951,6.027790e-16,-9.559995e-16,-0.040747


In [9]:
import pickle

# Guardar el modelo para el Cluster 0
with open("modelo_random_forest_cluster_0.pkl", "wb") as file:
    pickle.dump(model_0, file)

# Guardar el modelo para el Cluster 1
with open("modelo_random_forest_cluster_1.pkl", "wb") as file:
    pickle.dump(model_1, file)

# Guardar el modelo de KMeans
with open("modelo_kmeans.pkl", "wb") as file:
    pickle.dump(kmeans, file)

print("Modelos guardados exitosamente.")


Modelos guardados exitosamente.


In [10]:
# Crear una lista única de todas las variables sin duplicados
todas_las_variables = list(set(variables_importantes_captación + variables_importantes_cruda + variables_importantes_mez))

# Si deseas ordenarlas, puedes hacerlo para facilidad de revisión
todas_las_variables.sort()

# Confirmar que no hay duplicados y revisar el total de variables seleccionadas
print(f"Total de variables después de eliminar duplicados: {len(todas_las_variables)}")

Total de variables después de eliminar duplicados: 49


In [11]:
# En el entrenamiento original, asegúrate de que `todas_las_variables` esté en el orden correcto.
todas_las_variables = variables_importantes_captación + variables_importantes_cruda + variables_importantes_mez

# Asegúrate de que `X` al entrenar el modelo `KMeans` use el orden correcto
X = df_standardized_global[todas_las_variables]
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

# Guarda el modelo `KMeans` nuevamente si lo entrenas
with open("modelo_kmeans.pkl", "wb") as f:
    pickle.dump(kmeans, f)

C:\Users\Sofia\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [12]:
print("Orden de columnas en el entrenamiento:", todas_las_variables)

Orden de columnas en el entrenamiento: ['ALUMINIO_RESIDUAL_MG_L_AL_BN', 'CLORUROS_MG_L_CL_BN', 'SULFATOS_MG_L_SO4_BN', 'CONDUCTIVIDAD_US_CM_BN', 'OXIGENO_DISUELTO_MG_L_O2_BN', 'HIERRO_TOTAL_MG_L_FE_3_BN', 'TEMPERATURA_C_BN', 'NITRATOS_MG_L_NO3_BN', 'NITROGENO_AMONIACAL_G_L_BN', 'NITRITOS_MG_L_NO2_BN', 'MANGANESOS_MG_L_MN_BN', 'COT_MG_L_BN', 'ALCALINIDAD_TOTAL_MG_L_CACO3_BN', 'COLOR_UPC_BN', 'DUREZA_TOTAL_MG_L_CACO3_BN', 'MATERIA_ORGANICA_MG_L_BN', 'TURBIEDAD_UNT_BN', 'PH_BN', 'POTENCIAL_REDOX_MV_BN', 'DUREZA_CALCICA_MG_L_CACO3_BN', 'FOSFATOS_MG_L_BN', 'POTENCIAL_REDOX_MV_CRU', 'SULFATOS_MG_L_SO4_CRU', 'SOLIDOS_SUSPENDIDOS_MG_L_CRU', 'COT_MG_L_CRU', 'ALCALINIDAD_TOTAL_MG_L_CACO3_CRU', 'DUREZA_CALCICA_MG_L_CACO3_CRU', 'MANGANESOS_MG_L_MN_CRU', 'OXIGENO_DISUELTO_MG_L_O2_CRU', 'NITRATOS_MG_L_NO3_CRU', 'MATERIA_ORGANICA_MG_L_CRU', 'CONDUCTIVIDAD_US_CM_CRU', 'NITROGENO_AMONIACAL_G_L_CRU', 'TEMPERATURA_C_CRU', 'TURBIEDAD_UNT_CRU', 'NITRITOS_MG_L_NO2_CRU', 'FOSFATOS_MG_L_CRU', 'DUREZA_TOTAL_MG